# CTM MILP Formulation

In [1]:
import docplex.mp.model as cpx
import pandas as pd
import seaborn as sns
import random
from pprint import pprint

## Constants

In [2]:
# Constants from the parent paper

TOTAL_CELL_COUNT    = 32
MOVEMENT_CELLS      = 3 # Number of movement cells per approach
APPROACH_CELLS      = 3 # Number of cells in each approach
APPROACHES          = 4 # Number of approaches per intersection
APPROACH_LANES      = 4 # Number of lanes per approach

FREE_FLOW_SPEED     = 44 # ft / s
CELL_LENGTH         = 88 # ft
SAT_FLOW_RATE       = 1 # vehicles / timestep
FLOW_RATE_REDUCTION = 0.5 # Not speciifed
G_MIN               = 6 # seconds (change to 30)
G_MAX               = 20 # seconds (change to 120)

FLOW_UNDERSAT       = 450 # veh / hr / lane
FLOW_SAT            = 900 
FLOW_OVERSAT        = 1800

TURN_RATIO_LEFT     = 0.1
TURN_RATIO_THROUGH  = 0.8
TURN_RATIO_RIGHT    = 0.1
TURN_RATIOS = [
    TURN_RATIO_LEFT,
    TURN_RATIO_THROUGH,
    TURN_RATIO_RIGHT
]

LEFT_TURN_LANES     = 1
RIGHT_TURN_LANES    = 1
THROUGH_TURN_LANES  = APPROACH_LANES - LEFT_TURN_LANES - RIGHT_TURN_LANES
if THROUGH_TURN_LANES <= 0:
    THROUGH_TURN_LANES = 1
TURN_LANES = [
    LEFT_TURN_LANES,
    THROUGH_TURN_LANES,
    RIGHT_TURN_LANES
]

TIME_STEP           = 1 # seconds / time step; NOT FROM PAPER
TIME_RANGE          = 60 # run for this many seconds

MEAN_CAR_LENGTH     = 15.8 # ft

CELL_SOURCE         = 0
CELL_SINK           = 1
CELL_MOVEMENT       = 2
CELL_NORMAL         = 3

LEFT_TURN           = 0
THROUGH_TURN        = 1
RIGHT_TURN          = 2

## Sets

In [3]:
set_T = range(TIME_RANGE)
set_T_bounded = range(TIME_RANGE-1)

# Source cells: (0,approach_id)
set_C_O = [(CELL_SOURCE,0,i)
    for i in range(APPROACHES)]

# Sink cells: (0,approach_id)
set_C_S = [(CELL_SINK,0,i)
    for i in range(APPROACHES)]

# Movement cells: (movement_id, apporach_id)
set_C_I = [(CELL_MOVEMENT,i,j)
    for i in range(MOVEMENT_CELLS)
    for j in range(APPROACHES)]

# Normal cells: (cell_id, approach_id)
set_C_N = [(CELL_NORMAL,i,j)
    for i in range(APPROACH_CELLS)
    for j in range(APPROACHES)]

# Set of all cells: (cell_type, x, y)
set_C = set_C_O + set_C_S + set_C_I + set_C_N
set_C_labels = [
    'source',
    'sink',
    'movement',
    'normal'
]

In [4]:
def P_mapping(i):
    # 1. For source cells, return empty set
    if i[0] == CELL_SOURCE:
        return []
    # 2. For sink cells, return movement cells that lead to sink cell
    if i[0] == CELL_SINK:
        output = []
        # Add left turn of right approach
        output.append((CELL_MOVEMENT,LEFT_TURN,(i[2]+3)%4))
        # Add through turn of front approach
        output.append((CELL_MOVEMENT,THROUGH_TURN,(i[2]+2)%4))
        # Add right turn of left approach
        output.append((CELL_MOVEMENT,RIGHT_TURN,(i[2]+1)%4))
        return output
    # 3. For movement cells, return the previous cell
    if i[0] == CELL_MOVEMENT:
        return [(CELL_NORMAL,APPROACH_CELLS-1,i[2])]
    # 4. For normal cells, return the previous cell
    if i[0] == CELL_NORMAL:
        if i[1] == 0:
            return [(CELL_SOURCE,0,i[2])]
        else:
            return [(CELL_NORMAL,i[1]-1,i[2])]

def S_mapping(i):
    # 1. For source cells, return the next cell
    if i[0] == CELL_SOURCE:
        return [(CELL_NORMAL,0,i[2])]
    # 2. For sink cells, return empty set
    if i[0] == CELL_SINK:
        return []
    # 3. For movement cells, return the sink cell
    if i[0] == CELL_MOVEMENT:
        return [(CELL_SINK,0,(i[2]+i[1]+1)%4)]
    # 4. For normal cells, return the next cell/s
    if i[0] == CELL_NORMAL:
        if i[1] == APPROACH_CELLS-1:
            return [(CELL_MOVEMENT,x,i[2]) for x in range(MOVEMENT_CELLS)]
        else:
            return [(CELL_NORMAL,i[1]+1,i[2])]

def J_mapping(i):
    # Only for movement cells
    if i[0] == CELL_MOVEMENT:
        # Rights conflict only the left approach's Through
        if i[1] == RIGHT_TURN:
            return [(CELL_MOVEMENT, THROUGH_TURN, (i[2]+1)%4)]
        # Throughs conflict a lot of things
        if i[1] == THROUGH_TURN:
            output = []
            output = output + [
                (CELL_MOVEMENT, LEFT_TURN, (i[2]+2)%4)
            ]
            output = output + [
                (CELL_MOVEMENT, LEFT_TURN, (i[2]+3)%4),
                (CELL_MOVEMENT, THROUGH_TURN, (i[2]+3)%4),
                (CELL_MOVEMENT, RIGHT_TURN, (i[2]+3)%4)
            ]
            output = output + [
                (CELL_MOVEMENT, LEFT_TURN, (i[2]+1)%4),
                (CELL_MOVEMENT, THROUGH_TURN, (i[2]+1)%4)
            ]
            return output
        if i[1] == LEFT_TURN:
            output = []
            output = output + [
                (CELL_MOVEMENT, THROUGH_TURN, (i[2]+2)%4)
            ]
            output = output + [
                (CELL_MOVEMENT, LEFT_TURN, (i[2]+3)%4),
                (CELL_MOVEMENT, THROUGH_TURN, (i[2]+3)%4)
            ]
            output = output + [
                (CELL_MOVEMENT, LEFT_TURN, (i[2]+1)%4),
                (CELL_MOVEMENT, THROUGH_TURN, (i[2]+1)%4)
            ]
            return output
            
        
P = {i: P_mapping(i)
    for i in set_C}

S = {i: S_mapping(i)
    for i in set_C}

J = {i: J_mapping(i)
    for i in set_C_I}

In [5]:
pprint(J)

{(2, 0, 0): [(2, 1, 2), (2, 0, 3), (2, 1, 3), (2, 0, 1), (2, 1, 1)],
 (2, 0, 1): [(2, 1, 3), (2, 0, 0), (2, 1, 0), (2, 0, 2), (2, 1, 2)],
 (2, 0, 2): [(2, 1, 0), (2, 0, 1), (2, 1, 1), (2, 0, 3), (2, 1, 3)],
 (2, 0, 3): [(2, 1, 1), (2, 0, 2), (2, 1, 2), (2, 0, 0), (2, 1, 0)],
 (2, 1, 0): [(2, 0, 2), (2, 0, 3), (2, 1, 3), (2, 2, 3), (2, 0, 1), (2, 1, 1)],
 (2, 1, 1): [(2, 0, 3), (2, 0, 0), (2, 1, 0), (2, 2, 0), (2, 0, 2), (2, 1, 2)],
 (2, 1, 2): [(2, 0, 0), (2, 0, 1), (2, 1, 1), (2, 2, 1), (2, 0, 3), (2, 1, 3)],
 (2, 1, 3): [(2, 0, 1), (2, 0, 2), (2, 1, 2), (2, 2, 2), (2, 0, 0), (2, 1, 0)],
 (2, 2, 0): [(2, 1, 1)],
 (2, 2, 1): [(2, 1, 2)],
 (2, 2, 2): [(2, 1, 3)],
 (2, 2, 3): [(2, 1, 0)]}


## Parameters

In [6]:
def M_mapping(i):
    if i in set_C_I:
        return (CELL_LENGTH / MEAN_CAR_LENGTH) * TURN_LANES[i[1]]
    elif i in set_C_O:
        return float("inf")
    return (CELL_LENGTH / MEAN_CAR_LENGTH) * APPROACH_LANES

def F_mapping(i):
    if i in set_C_I:
        return SAT_FLOW_RATE * TURN_LANES[i[1]]
    return SAT_FLOW_RATE * APPROACH_LANES

In [7]:
d = {(i,t): FLOW_SAT*APPROACH_LANES*TIME_STEP / (3600)
    for i in set_C_O
    for t in set_T}

M = {i: M_mapping(i)
    for i in set_C}

F = {i: F_mapping(i)
    for i in set_C}

r = {i: TURN_RATIOS[i[1]]
    for i in set_C_I}

alpha = 0.8

## Initialize Model

In [8]:
model = cpx.Model(name="Thesis MILP model")

## Decision Variables

In [9]:
g_vars = {(i,t): model.binary_var(
    name="g_{}^{}".format(i,t))
for i in set_C_I
for t in set_T}

In [10]:
x_vars = {(i,t): model.continuous_var(
    lb=0,
    ub=M[i],
    name="x_{}^{}".format(i,t))
for i in set_C
for t in set_T}

In [11]:
y_vars = {(i,j,t): model.continuous_var(
    lb=0,
    ub=min(F[i],F[j]),
    name="y_{}_{}^{}".format(i,j,t))
for i in set_C
for j in S_mapping(i)
for t in set_T}

## Constraints

### Initial Values

In [12]:
init_src = [
    (model.add_constraint(
        ct=(
            x_vars[(i,0)]
            == d[(i,0)]
        ),
        ctname="init_src_{}".format(i)
    ))
    for i in set_C_O
]

In [13]:
constraint_init = {
    'src': init_src
}

### Flow Conservation

In [14]:
flowcon_1 = [
    (model.add_constraint(
        ct=(
            model.sum(y_vars[(k,i,t)] for k in P[i])
            - model.sum(y_vars[(i,j,t)] for j in S[i])
            - x_vars[(i,t+1)]
            + x_vars[(i,t)]
            == 0
        ),
        ctname="flowcon_normal_{}^{}".format(i,t)
    ))
    for t in set_T_bounded
    for i in set_C_N + set_C_I
]

In [15]:
flowcon_2 = [
    (model.add_constraint(
        ct=(
            d[(i,t)]
            - model.sum(y_vars[(i,j,t)] for j in S[i])
            - x_vars[(i,t+1)]
            + x_vars[(i,t)]
            == 0
        ),
        ctname="flowcon_source_{}^{}".format(i,t)
    ))
    for t in set_T_bounded
    for i in set_C_O
]

In [16]:
flowcon_3 = [
    (model.add_constraint(
        ct=(
            model.sum(y_vars[(k,i,t)] for k in P[i])
            - x_vars[(i,t+1)]
            + x_vars[(i,t)]
            == 0
        ),
        ctname="flowcon_sink_{}^{}".format(i,t)
    ))
    for t in set_T_bounded
    for i in set_C_S
]

In [17]:
constraint_flowcon = {
    'source': flowcon_2,
    'sink': flowcon_3,
    'rest': flowcon_1
}

print("Expected constraint count: {}".format((TIME_RANGE-1) * TOTAL_CELL_COUNT))
print("Actual constraint count: {}".format(len(flowcon_1) + len(flowcon_2) + len(flowcon_3)))


Expected constraint count: 1888
Actual constraint count: 1888


### Flow Rate

In [18]:
flowrate_1 = [
    (model.add_constraint(
        ct=(
            model.sum(y_vars[(i,j,t)] for j in S[i])
            - x_vars[(i,t)]
            <= 0
        ),
        ctname="flowrate_srccap_{}^{}".format(i,t)
    ))
    for t in set_T
    for i in set_C if i not in set_C_S
]

In [19]:
flowrate_2 = [
    (model.add_constraint(
        ct=(
            model.sum(y_vars[(i,j,t)] for i in P[j])
            - M[j]
            + x_vars[(j,t)]
            <= 0
        ),
        ctname="flowrate_destcap_{}^{}".format(j,t)
    ))
    for t in set_T
    for j in set_C if j not in set_C_O
]

In [20]:
constraint_flowrate = {
    'source_cap': flowrate_1,
    'sink_cap': flowrate_2
}

print("Expected constraint count: {}".format(TIME_RANGE * (TOTAL_CELL_COUNT-4) * 2))
print("Actual constraint count: {}".format(len(flowrate_1) + len(flowrate_2)))

Expected constraint count: 3360
Actual constraint count: 3360


### Turning Ratios

In [21]:
turnratios = [
    (model.add_constraint(
        ct=(
            y_vars[(i,j,t)]
            - model.sum(r[j] * y_vars[(i,k,t)] for k in S[i])
            <= 0
        ),
        ctname="turnratios_{},{}^{}".format(i,j,t)
    ))
    for t in set_T
    for j in set_C_I
    for i in P[j]
]

In [22]:
constraint_turnratios = {
    'turn_ratios': turnratios
}

print("Expected constraint count: {}".format(TIME_RANGE * MOVEMENT_CELLS * APPROACHES))
print("Actual constraint count: {}".format(len(turnratios)))

Expected constraint count: 720
Actual constraint count: 720


### Movement Cell Flow Rate

In [23]:
green_flowrate = [
    (model.add_constraint(
        ct=(
            y_vars[(i,j,t)]
            - F[i]*g_vars[(i,t)]
            <= 0
        ),
        ctname="green_flowrate_{},{}^{}".format(i,j,t)
    ))
    for t in set_T
    for i in set_C_I
    for j in S[i]
]

In [24]:
slowstart_flowrate = [
    (model.add_constraint(
        ct=(
            y_vars[(i,j,t)]
            - F[i]
            + (F[i]*FLOW_RATE_REDUCTION)*g_vars[(i,t+1)]
            - (F[i]*FLOW_RATE_REDUCTION)*g_vars[(i,t)]
            <= 0
        ),
        ctname="slowstart_flowrate_{},{}^{}".format(i,j,t)
    ))
    for t in set_T_bounded
    for i in set_C_I
    for j in S[i]
]

In [25]:
constraint_greenflowrate = {
    'green_flowrate': green_flowrate,
    'slowstart_flowrate': slowstart_flowrate
}

print("Expected constraint count: {}".format((TIME_RANGE * MOVEMENT_CELLS * APPROACHES) + ((TIME_RANGE-1) * MOVEMENT_CELLS * APPROACHES)))
print("Actual constraint count: {}".format(len(green_flowrate) + len(slowstart_flowrate)))

Expected constraint count: 1428
Actual constraint count: 1428


### Green Time Limit

In [26]:
green_max = [
    (model.add_constraint(
        ct=(
            model.sum(g_vars[(i,z)] for z in range(t,t+G_MAX+2))
            - G_MAX*TIME_STEP
            <= 0
        ),
        ctname='green_max_{}^{}'.format(i,t)
    ))
    for t in range(TIME_RANGE - G_MAX - 1)
    for i in set_C_I
]

In [27]:
green_min = [
    (model.add_constraint(
        ct=(
            model.sum(g_vars[(i,z)] for z in range(t+1,t+G_MIN+1))
            - G_MIN*g_vars[(i,t+1)]
            + G_MIN*g_vars[(i,t)]
            >= 0
        ),
        ctname='green_min_{}^{}'.format(i,t)
    ))
    for t in range(TIME_RANGE - G_MIN)
    for i in set_C_I
]

In [28]:
constraint_greentime = {
    'green_max': green_max,
    'green_min': green_min
}

print("Expected constraint count: {}".format(((TIME_RANGE - G_MAX - 1) * MOVEMENT_CELLS * APPROACHES) + ((TIME_RANGE - G_MIN) * MOVEMENT_CELLS * APPROACHES)))
print("Actual constraint count: {}".format(len(green_max) + len(green_min)))

Expected constraint count: 1116
Actual constraint count: 1116


### Conflicting Movements

In [29]:
movements_min = [
    (model.add_constraint(
        ct=(
            model.sum(g_vars[(i,t)] for i in set_C_I)
            >= 2
        ),
        ctname='movements_min^{}'.format(t)
    ))
    for t in set_T
]

In [30]:
movements_max = [
    (model.add_constraint(
        ct=(
            model.sum(g_vars[(i,t)] for i in set_C_I)
            <= 4
        ),
        ctname='movements_max^{}'.format(t)
    ))
    for t in set_T
]

In [31]:
movements_conflicting = [
    (model.add_constraint(
        ct=(
            g_vars[(i,t)]
            + g_vars[(j,t)]
            <= 1
        ),
        ctname='movements_conflicting_{},{}^{}'.format(i,j,t)
    ))
    for t in set_T
    for i in set_C_I
    for j in J[i]
]

In [32]:
constraint_conflicts = {
    'movements_min': movements_min,
    'movements_max': movements_max,
    'movements_conflicting': movements_conflicting
}

print("Expected constraint count: {}".format('???'))
print("Actual constraint count: {}".format(len(green_max) + len(green_min)))

Expected constraint count: ???
Actual constraint count: 1116


### Collate all the constraints

In [33]:
constraints = {
    'flowcon': constraint_flowcon,
    'flowrate': constraint_flowrate,
    'turnratios': constraint_turnratios,
    'greenflowrate': constraint_greenflowrate,
    'greentime': constraint_greentime,
    'conflicts': constraint_conflicts,
    'init': constraint_init
}

total_constraints = 0

for _, constraint_dict in constraints.iteritems():
    for _, constraint_array in constraint_dict.iteritems():
        total_constraints = total_constraints + len(constraint_array)
        
print("Total constraint count: {}".format(total_constraints))

Total constraint count: 11516


## Objective Function

In [34]:
D_max = 1.0 / sum([ M[i] for i in set_C for t in set_T ])
T_max = 1.0 / sum([ M[i] for i in set_C_S for t in set_T ])

D = model.sum(
    model.sum(
        D_max * x_vars[(i,t)] - model.sum(
            D_max * y_vars[(i,j,t)]
            for j in S_mapping(i))
        for i in set_C)
    for t in set_T)

T = model.sum(
    model.sum(
        T_max * x_vars[(i,t)]
        for i in set_C_S)
    for t in set_T)


objective = alpha*D - (1-alpha)*T

In [35]:
model.minimize(objective)

## Solving

In [36]:
print("Solving...")
model.solve()
print("Done!")

Solving...
Done!


## Results

In [37]:
opt_df = pd.DataFrame.from_dict(g_vars, orient="index", 
                                columns = ["variable_object"])

opt_df.reset_index(inplace=True)

opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.solution_value)

print(opt_df)
print(model.get_solve_details())
print(model.objective_value)
print(opt_df.dtypes)

               index variable_object  solution_value
0    ((2, 2, 1), 57)  g_(2, 2, 1)^57             0.0
1    ((2, 1, 3), 17)  g_(2, 1, 3)^17             0.0
2    ((2, 2, 3), 59)  g_(2, 2, 3)^59             1.0
3    ((2, 0, 1), 40)  g_(2, 0, 1)^40             0.0
4    ((2, 2, 2), 58)  g_(2, 2, 2)^58             0.0
5    ((2, 1, 1), 41)  g_(2, 1, 1)^41             0.0
6    ((2, 0, 3), 14)  g_(2, 0, 3)^14             0.0
7    ((2, 1, 3), 11)  g_(2, 1, 3)^11             0.0
8    ((2, 2, 3), 37)  g_(2, 2, 3)^37             1.0
9    ((2, 1, 0), 44)  g_(2, 1, 0)^44             0.0
10   ((2, 2, 2), 28)  g_(2, 2, 2)^28             1.0
11   ((2, 0, 1), 28)  g_(2, 0, 1)^28             0.0
12   ((2, 0, 3), 20)  g_(2, 0, 3)^20             0.0
13   ((2, 1, 3), 37)  g_(2, 1, 3)^37             0.0
14   ((2, 2, 0), 58)  g_(2, 2, 0)^58             1.0
15   ((2, 0, 1), 30)  g_(2, 0, 1)^30             0.0
16   ((2, 1, 0), 10)  g_(2, 1, 0)^10             0.0
17   ((2, 1, 2), 56)  g_(2, 1, 2)^56          

## Showing the Green Light Phasing

In [38]:
df_g_raw = pd.DataFrame.from_dict(g_vars, orient="index", 
                                columns = ["variable_object"])

df_g_raw.reset_index(inplace=True)
df_g_raw["is_green"] = df_g_raw["variable_object"].apply(lambda item: item.solution_value)
df_g_raw['cell'] = df_g_raw['index'].apply(lambda x: x[0])
df_g_raw['timestep'] = df_g_raw['index'].apply(lambda x: x[1])

df_g = df_g_raw[['timestep', 'cell', 'is_green']]
df_g

,timestep,cell,is_green
0,57,"(2, 2, 1)",0.0
1,17,"(2, 1, 3)",0.0
2,59,"(2, 2, 3)",1.0
3,40,"(2, 0, 1)",0.0
4,58,"(2, 2, 2)",0.0
5,41,"(2, 1, 1)",0.0
6,14,"(2, 0, 3)",0.0
7,11,"(2, 1, 3)",0.0
8,37,"(2, 2, 3)",1.0
9,44,"(2, 1, 0)",0.0


In [39]:
df_g.cell.value_counts()

(2, 2, 0)    60
(2, 1, 1)    60
(2, 2, 1)    60
(2, 1, 0)    60
(2, 2, 2)    60
(2, 1, 3)    60
(2, 2, 3)    60
(2, 1, 2)    60
(2, 0, 2)    60
(2, 0, 3)    60
(2, 0, 0)    60
(2, 0, 1)    60
Name: cell, dtype: int64

In [40]:
df_g_specific = df_g[df_g.cell == (2,1,3)].sort_values(by='timestep')
df_g_specific

,timestep,cell,is_green
254,0,"(2, 1, 3)",0.0
561,1,"(2, 1, 3)",0.0
145,2,"(2, 1, 3)",0.0
456,3,"(2, 1, 3)",0.0
34,4,"(2, 1, 3)",0.0
338,5,"(2, 1, 3)",0.0
643,6,"(2, 1, 3)",0.0
230,7,"(2, 1, 3)",0.0
535,8,"(2, 1, 3)",0.0
120,9,"(2, 1, 3)",0.0


## Showing the Cell Capacities

In [41]:
df_x_raw = pd.DataFrame.from_dict(x_vars, orient="index", 
                                columns = ["variable_object"])

df_x_raw.reset_index(inplace=True)
df_x_raw["volume"] = df_x_raw["variable_object"].apply(lambda item: item.solution_value)
df_x_raw['cell'] = df_x_raw['index'].apply(lambda x: x[0])
df_x_raw['timestep'] = df_x_raw['index'].apply(lambda x: x[1])

df_x = df_x_raw[['timestep', 'cell', 'volume']]
df_x

,timestep,cell,volume
0,57,"(2, 2, 1)",0.000000
1,3,"(3, 0, 3)",0.000000
2,26,"(1, 0, 3)",22.278481
3,41,"(2, 1, 1)",0.000000
4,6,"(3, 2, 3)",0.000000
5,37,"(2, 2, 3)",0.000000
6,51,"(0, 0, 2)",52.000000
7,37,"(2, 1, 3)",0.000000
8,26,"(1, 0, 1)",22.278481
9,26,"(3, 2, 0)",0.000000


In [43]:
df_x_specific = df_x[df_x.cell == (CELL_NORMAL,1,1)].sort_values(by='timestep')
df_x_specific

,timestep,cell,volume
86,0,"(3, 1, 1)",0.0
363,1,"(3, 1, 1)",0.0
642,2,"(3, 1, 1)",0.0
917,3,"(3, 1, 1)",0.0
899,4,"(3, 1, 1)",0.0
1175,5,"(3, 1, 1)",0.0
1455,6,"(3, 1, 1)",0.0
1728,7,"(3, 1, 1)",0.0
384,8,"(3, 1, 1)",0.0
663,9,"(3, 1, 1)",0.0
